In [45]:
#import dependencies 
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt

In [46]:
#import tensorflow dependencies- functional api
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf


In [47]:
#Siamese distance layer
class L1Dist(Layer):
    def __init__(self, **kwargs):
        super().__init__()
        
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

In [48]:
#reload model
facemodel = tf.keras.models.load_model('facesiamesemodel.h5', custom_objects={'L1Dist':L1Dist, 'BinaryCrossentropy':tf.losses.BinaryCrossentropy})

fpmodel = tf.keras.models.load_model('fpsiamesemodel.h5', custom_objects={'L1Dist':L1Dist, 'BinaryCrossentropy':tf.losses.BinaryCrossentropy})

In [49]:
#face preprocessing function
def facepreprocess(file_path):
    byte_img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(byte_img)
    img = tf.image.resize(img, (100,100))
    img = img/255.0
    return img

#fingerprint preprocessing function
def fppreprocess(file_path):
    byte_img = tf.io.read_file(file_path) 
    img = tf.io.decode_bmp(byte_img, channels=3)
    img = tf.image.resize(img, (100,100))
    img = img/255.0
    return img
    

## OpenCV Img capture

In [50]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    frame = frame[120:120+250,200:200+250, :]
    
    cv2.imshow('verification', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('v'):
        cv2.imwrite(os.path.join('face_input_images', 'input_image.jpg'), frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

## Face Verification function

In [51]:
def faceverify(facemodel, detection_threshold, verification_threshold):
   
    #verification Threshold: proportion of positive predictions/ total positive samples
    results = []
    for image in os.listdir('faceverification_images'):
        input_img = facepreprocess('./face_input_images/input_image.jpg')
        validation_img = facepreprocess(os.path.join('faceverification_images', image))
        
        # make predictions
        result = facemodel.predict(list(np.expand_dims([input_img, validation_img], axis=1)))
        results.append(result)
    
    #detection Threshold: metric above which a prediction is considered positive   
    detection = np.sum(np.array(results) > detection_threshold)
    #verification Threshold: proportion of positive predictions/ total positive samples
    verification = detection / len(os.listdir('faceverification_images'))
    verified = verification > verification_threshold
    
    return results, verified

## Run Face Verification

In [52]:
results, faceverified = faceverify(facemodel, 0.5, 0.5)
print(faceverified)


1/1 [==============================] - 0s 25ms/step
True


## Fingerprint verification function

In [33]:
def fpverify(fpmodel, detection_threshold, verification_threshold):
   
    #verification Threshold: proportion of positive predictions/ total positive samples
    results = []
    for image in os.listdir('fpverification_images'):
        input_img = fppreprocess('./fpinput/input_image.jpg')
        validation_img = fppreprocess(os.path.join('fpverification_images', image))
        
        # make predictions
        result = fpmodel.predict(list(np.expand_dims([input_img, validation_img], axis=1)))
        results.append(result)
    
    #detection Threshold: metric above which a prediction is considered positive   
    detection = np.sum(np.array(results) > detection_threshold)
    #verification Threshold: proportion of positive predictions/ total positive samples
    verification = detection / len(os.listdir('fpverification_images'))
    verified = verification > verification_threshold
    
    return results, verified

## Run fingerprint verification

In [34]:
results, fpverified = fpverify(fpmodel, 0.4, 0.4)
print(fpverified)


1/1 [==============================] - 0s 15ms/step
True


## AND Gate

In [35]:
 def AND (a, b):
 
    if a == True and b == True:
        return True
    else:
        return False
 

In [37]:
final_verified = AND(faceverified, fpverified)
print('The verification status is:', final_verified )

The verification status is: True
